# Explore Metadata Options

In [1]:
from typing import Dict

import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Polygon

from lonboard.colormap import apply_continuous_cmap
from lonboard import Map, ScatterplotLayer, PolygonLayer
from palettable.cartocolors.sequential import BurgYl_2
from geojson_pydantic import Feature 
from geojson_pydantic import Polygon as Polygon_

import h3
from shapely.geometry import Polygon, Point

## Local Data

In [4]:
df = pd.read_parquet("../../postgres/space2stats.parquet")

In [13]:
gdf = df.copy()

In [14]:
len(gdf)

14117882

In [15]:
gdf.loc[:, 'geometry'] = gdf.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x['hex_id'], geo_json=True)), axis=1)

In [17]:
gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs='EPSG:4326')

In [18]:
gdf.total_bounds

array([-179.99999562,  -89.98750455,  179.99999096,   89.98750455])

## Load Data from API Demo

In [19]:
BASE_URL = "https://space2stats.ds.io"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [20]:
response = requests.get(FIELDS_ENDPOINT)
if response.status_code != 200:
    raise Exception(f"Failed to get fields: {response.text}")

available_fields = response.json()
print("Available Fields:", available_fields)

Available Fields: ['sum_pop_2020', 'ogc_fid', 'sum_pop_f_0_2020', 'sum_pop_f_10_2020', 'sum_pop_f_15_2020', 'sum_pop_f_1_2020', 'sum_pop_f_20_2020', 'sum_pop_f_25_2020', 'sum_pop_f_30_2020', 'sum_pop_f_35_2020', 'sum_pop_f_40_2020', 'sum_pop_f_45_2020', 'sum_pop_f_50_2020', 'sum_pop_f_55_2020', 'sum_pop_f_5_2020', 'sum_pop_f_60_2020', 'sum_pop_f_65_2020', 'sum_pop_f_70_2020', 'sum_pop_f_75_2020', 'sum_pop_f_80_2020', 'sum_pop_m_0_2020', 'sum_pop_m_10_2020', 'sum_pop_m_15_2020', 'sum_pop_m_1_2020', 'sum_pop_m_20_2020', 'sum_pop_m_25_2020', 'sum_pop_m_30_2020', 'sum_pop_m_35_2020', 'sum_pop_m_40_2020', 'sum_pop_m_45_2020', 'sum_pop_m_50_2020', 'sum_pop_m_55_2020', 'sum_pop_m_5_2020', 'sum_pop_m_60_2020', 'sum_pop_m_65_2020', 'sum_pop_m_70_2020', 'sum_pop_m_75_2020', 'sum_pop_m_80_2020', 'sum_pop_m_2020', 'sum_pop_f_2020']


In [21]:
AOIModel = Feature[Polygon_, Dict]

# kenya
aoi = {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [41.85508309264397, -1.68325],
            [40.98105, -2.49979],
            [40.993, -3.444],
            [41.58513, -3.91909],
            [40.88477, -4.95913],
            [39.55938425876585, -4.437641590288629],
            [39.25451, -3.42206],
            [37.7669, -3.67712],
            [37.69869, -3.09699],
            [34.07262, -1.05982],
            [33.90371119710453, -0.95],
            [33.893568969666944, 0.109813537861896],
            [34.18, 0.515],
            [34.6721, 1.17694],
            [35.03599, 1.90584],
            [34.59607, 3.05374],
            [34.47913, 3.5556],
            [35.298007118232946, 4.77696566346189],
            [35.817447662353516, 4.77696566346189],
            [36.159078632855646, 4.447864127672769],
            [36.85509323800812, 4.447864127672769],
            [38.120915, 3.598605],
            [38.43697, 3.58851],
            [38.67114, 3.61607],
            [38.89251, 3.50074],
            [39.55938425876585, 3.42206],
            [39.85494, 3.83879],
            [40.76848, 4.25702],
            [41.1718, 3.91909],
            [41.85508309264397, 2.97959],
            [41.58513, 2.09],
            [40.993, 1.657],
            [40.98105, 1.002],
            [41.85508309264397, -1.68325]
          ]
        ]
      },
      "properties": {
        "name": "Kenya"
      }
    }


feat = AOIModel(**aoi)

In [22]:
# Define the Request Payload
request_payload = {
    "aoi": aoi,
    "spatial_join_method": "centroid",
    "fields": available_fields[0:5],  # fails if all fields are requested
    "geometry": "point"
}

# Get Summary Data
response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
if response.status_code != 200:
    raise Exception(f"Failed to get summary: {response.text}")

summary_data = response.json()
df = pd.DataFrame(summary_data)

In [24]:
response.

'utf-8'

In [9]:
df.head()

,hex_id,geometry,sum_pop_2020,ogc_fid,sum_pop_f_0_2020,sum_pop_f_10_2020,sum_pop_f_15_2020
0,866a4a48fffffff,"{'type': 'Point', 'coordinates': [36.098439120...",399.860905,6157365,4.405487,29.409952,26.127678
1,866a4a497ffffff,"{'type': 'Point', 'coordinates': [38.217348483...",582.555159,6157366,6.418330,42.847198,38.065269
2,866a4a49fffffff,"{'type': 'Point', 'coordinates': [40.203833582...",749.911237,6157367,8.262184,55.156311,49.000641
3,866a4a4d7ffffff,"{'type': 'Point', 'coordinates': [35.196193074...",863.888290,6157373,11.218388,64.547882,49.363289
4,866a5820fffffff,"{'type': 'Point', 'coordinates': [36.570238767...",525.085147,6161418,5.785151,38.620255,34.310074


In [10]:
gdf = df.copy()
gdf.loc[:, 'geometry'] = gdf['hex_id'].apply(lambda x: Polygon(h3.h3_to_geo_boundary(x, geo_json=True)))
gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs='epsg:4326')
out_file = "./test.geojson"
gdf.to_file(out_file, driver='GeoJSON')

## Create STAC

In [8]:
import shutil
import tempfile
from pathlib import Path

from pystac import Catalog, Item, Asset, CatalogType, get_stac_version
import fio_stac

In [9]:
print(get_stac_version())

1.0.0


### Catalog  

Basic description of project and dataset.

In [10]:
Catalog?

Init signature:
Catalog(
    id: 'str',
    description: 'str',
    title: 'str | None' = None,
    stac_extensions: 'list[str] | None' = None,
    extra_fields: 'dict[str, Any] | None' = None,
    href: 'str | None' = None,
    catalog_type: 'CatalogType' = ABSOLUTE_PUBLISHED,
    strategy: 'HrefLayoutStrategy | None' = None,
)
Docstring:     
A PySTAC Catalog represents a STAC catalog in memory.

A Catalog is a :class:`~pystac.STACObject` that may contain children,
which are instances of :class:`~pystac.Catalog` or :class:`~pystac.Collection`,
as well as :class:`~pystac.Item` s.

Args:
    id : Identifier for the catalog. Must be unique within the STAC.
    description : Detailed multi-line description to fully explain the catalog.
        `CommonMark 0.29 syntax <https://commonmark.org/>`_ MAY be used for rich
        text representation.
    title : Optional short descriptive one-line title for the catalog.
    stac_extensions : Optional list of extensions the Catalog implements.
 

In [11]:
catalog = Catalog(
    id="space2stats", 
    description="Global dataset of geospatial variables at the grid level (hexagon H3 level 5).",
    title="Space2Stats"
    )

### STAC Item

Can represent the global H3 parquet file with column descriptions for each variable.

In [12]:
descriptions = [
    "Hexagon unique identifier",
    "Geometry (shapely polygon) of hexagon",
    "Total population in 2020",
    "OGC FID",
    "Total female population, age 0-1",
    "Total female population, age 10-15",
    "Total female population, age 15-20"
]

data_dict = []
for column, description in zip(gdf.columns, descriptions):
    data_dict.append({
        "name": column,
        "description": description,
        "type": str(gdf[column].dtype),
        })

In [13]:
item = fio_stac.create_stac_item(
    source = out_file,
    id = "space2stats-population",
    extensions = ['https://stac-extensions.github.io/table/v1.2.0/schema.json'],
    properties = {
        "name": "Space2Stats Population",
        "description": "Testing metadata with population variables",
        "table:primary_geometry" : "geometry",
        "table:columns" : data_dict,
    }
)

In [14]:
item

<Item id=space2stats-population>

In [15]:
catalog.add_item(item)

<Link rel=item target=<Item id=space2stats-population>>

In [16]:
print(list(catalog.get_children()))
print(list(catalog.get_items()))

[]
[<Item id=space2stats-population>]


In [17]:
catalog.describe()

* <Catalog id=space2stats>
  * <Item id=space2stats-population>


### Asset

Link to World Bank metadata page with appropriate schema (DDH or NADA).  
Can store additional information about authors, the source for input data, how it was processed etc.

In [11]:
Asset?

Object `Asset` not found.


In [12]:
example_url = 'https://nada-demo.ihsn.org/index.php/catalog/55'

In [13]:
asset = Asset(
    href=example_url,
    title="Additional Medata",
    media_type="text/html",
    roles=["metadata"]
)

NameError: name 'Asset' is not defined

In [ ]:
item.add_asset("metadata", asset)

### Save Demo

In [24]:
from os.path import join

In [25]:
print(catalog.get_self_href() is None)
print(item.get_self_href() is None)

True
True


In [26]:
catalog.normalize_hrefs(join(".", "stac"))

In [27]:
print(catalog.get_self_href())
print(item.get_self_href())

c:/Users/WB514197/Repos/DECAT_Space2Stats/notebooks/METADATA/stac/catalog.json
c:/Users/WB514197/Repos/DECAT_Space2Stats/notebooks/METADATA/stac/space2stats-population/space2stats-population.json


In [28]:
catalog.save(catalog_type=CatalogType.SELF_CONTAINED)

## AWS List

aws s3 ls s3://wbg-geography01/Space2Stats/h3_stats_data/AFR/ --no-verify-ssl

In [2]:
import boto3
from botocore.config import Config
import pandas as pd

In [3]:
client = boto3.client('s3')
paginator = client.get_paginator('list_objects')

s3 = boto3.resource('s3')
bucket = s3.Bucket('wbg-geography01')

In [4]:
list(bucket.objects.filter(Prefix="Space2Stats/h3_stats_data/GLOBAL"))[0:5]

[s3.ObjectSummary(bucket_name='wbg-geography01', key='Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20230601-20230630_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv'),
 s3.ObjectSummary(bucket_name='wbg-geography01', key='Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20230701-20230731_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv'),
 s3.ObjectSummary(bucket_name='wbg-geography01', key='Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20230901-20230930_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv'),
 s3.ObjectSummary(bucket_name='wbg-geography01', key='Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20231001-20231031_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv'),
 s3.ObjectSummary(bucket_name='wbg-geography01', key='Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20231101-20231130_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv')]

In [5]:
result = paginator.paginate(Bucket='wbg-geography01', Prefix='Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff', Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/


In [6]:
result = paginator.paginate(Bucket='wbg-geography01', Prefix='Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/', Delimiter='/')
n = 0
for page in result:
    n += 1
    for item in page['Contents']:
        fname = item['Key']
        if fname.endswith('.csv'):
            print(fname)

Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20230601-20230630_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv
Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20230701-20230731_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv
Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20230901-20230930_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv
Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20231001-20231031_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv
Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20231101-20231130_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv
Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20231201-20231231_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv
Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20240101-20240131_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv
Space2Stats/h3_stats_data/G

In [7]:
df = pd.read_csv('s3://wbg-geography01/Space2Stats/h3_stats_data/GLOBAL/VIIRS_Monthly_LEN/8001fffffffffff/DNB_j01_20230601-20230630_global_ecm-slcorr_v10_ops.avg_rade9_zonal.csv')